## Useful libraries

In [273]:
import pandas as pd 
import numpy as np
import datetime 
from datetime import datetime as dt
import yfinance as yf
import warnings

## Downloading data

We only have to download data regarding daily prices of the choesen set of stocks since data about the S&P500 is easily available on FactSet

In [207]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# read in the url and scrape ticker data
data_table = pd.read_html(url)

In [208]:
tickers_in_SP500 = data_table[0]['Symbol'].tolist() #these are all SP500 tickers at the date of execution

Now we want to remove any ticker that got in or out in the S&P500; that is we want to keep only all those tickers that were present trhoughout the whole timeframe considered. So we have to create a list of tickers that got in/out of the index in said period

In [209]:
data_to_remove=data_table[1]
ticker_add_list = data_to_remove['Added']['Ticker']
ticker_rem_list = data_to_remove['Removed']['Ticker']
datalist = data_to_remove['Date']['Date']

Now we can define an useful function to convert our dates in datetime-like values

In [210]:
def convert(date_time): 
    format = '%B %d %Y'
    datetime_str = datetime.datetime.strptime(date_time, format) 
  
    return datetime_str

In [211]:
tickers_to_remove_frame=pd.DataFrame({'datalist': datalist, 
                                      'ticker_add_list': ticker_add_list, 
                                      'ticker_rem_list': ticker_rem_list})

And now we have to take care of commas in dates and convert:

In [212]:
tickers_to_remove_frame['datalist']=tickers_to_remove_frame['datalist'].str.replace(',', '').apply(convert)

Now we only want to keep track of unique tickers from 01-06-2017 onwards; moreover we are interested in just the ticker values

In [213]:
start_date = dt(2017, 6, 1)
end_date = dt(2024, 7, 2)

In [214]:
to_remove=set(list(tickers_to_remove_frame
                   [tickers_to_remove_frame['datalist']>=start_date]['ticker_add_list'])
              +list(tickers_to_remove_frame
                    [tickers_to_remove_frame['datalist']>=start_date]['ticker_rem_list']))

In [215]:
stable_tickers=set(tickers_in_SP500).difference(to_remove)

In [216]:
len(stable_tickers)

391

Now we know we have 391 publicly traded companies that were in the S&P500 and stayed in the S&P500 during our timeframe (1st June 2017-> 1st July 2024), but we should convert the set into a list since it's more useful (and we also take care of some typos in tickers' values, such as the fact that yfinance accepts "-" but not ".")

In [217]:
stable_ticker_list=list(stable_tickers)
stable_ticker_list.sort()
for i in range(len(stable_ticker_list)):
    stable_ticker_list[i]=stable_ticker_list[i].replace('.', '-')

Now we proceed to finally download data from yahoo finance. Yahoo finance is good enough since we are only interested in price data and nothing else (and its precision is sufficient for this)

In [218]:
# we first start by downloading the first element in stable_ticker_list, then we will concat everything in a 
# single big dataframe with tickers for columns and dates for rows. 
data = yf.download(stable_ticker_list[0], start=start_date, end=end_date)
data.drop(['Open', 'High', 'Close', 'Low', 'Volume'], axis=1, inplace=True) #dropping useless columns
data.reset_index(inplace=True)
data.rename({'Adj Close': stable_ticker_list[0]}, axis='columns', inplace=True) #inserting tickers' name as col
prices_stocks_list=pd.DataFrame(data) #create first draft of df, to be concatenated later
stable_ticker_list.remove(stable_ticker_list[0])

for ticker in stable_ticker_list:
    # now repeat same process for every ticker
    data = yf.download(ticker, start=start_date, end=end_date)
    data.drop(['Open', 'High', 'Close', 'Low', 'Volume'], axis=1, inplace=True)
    data.reset_index(inplace=True)
    data.drop('Date', inplace=True, axis=1)
    data.rename({'Adj Close': ticker}, axis='columns', inplace=True)
    # append the data to the prices DataFrame
    prices_stocks_list=pd.concat([prices_stocks_list, data], axis=1)
prices_stocks_list

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,A,AAL,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADM,...,WRK,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
0,2017-06-01,57.481789,47.606354,35.847012,49.100388,40.796951,112.791328,141.380005,71.465996,34.372223,...,44.562824,132.199722,24.966391,123.336929,38.687820,57.868073,48.211994,65.153488,112.562195,59.689930
1,2017-06-02,57.557507,48.062534,36.378246,49.497860,41.150944,113.196434,143.479996,70.335915,34.700985,...,44.425758,132.695572,24.966391,124.343285,38.840542,57.007584,48.726845,65.294693,113.727554,59.556530
2,2017-06-05,57.169434,48.276058,36.022530,49.689209,41.257133,113.952644,143.589996,68.458176,34.470848,...,44.062935,132.578384,24.869534,123.050720,38.454704,57.452171,48.910717,65.479988,113.718384,59.718517
3,2017-06-06,56.885483,48.276058,36.144222,49.954182,41.027046,113.727585,143.029999,69.631737,34.479061,...,43.635609,131.081772,24.891886,122.422920,38.213562,58.233776,48.441853,65.585907,113.938629,60.013920
4,2017-06-07,57.387127,49.363083,36.359520,50.601894,41.204037,113.925644,143.619995,70.663322,34.446201,...,44.941765,131.063690,24.831726,122.155190,38.502941,58.018658,49.149765,65.532951,114.103783,60.280735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,2024-06-25,134.440491,11.130000,209.070007,170.750000,105.489998,307.170013,526.880005,230.339996,60.650002,...,50.209999,262.193207,28.280001,88.300003,53.740002,114.370003,136.929993,132.330002,107.980003,167.979996
1778,2024-06-26,132.843460,11.120000,213.250000,171.149994,104.879997,304.619995,528.659973,227.839996,60.669998,...,49.700001,262.771271,27.959999,88.129997,53.790001,114.410004,136.210007,131.960007,108.110001,170.899994
1779,2024-06-27,130.557709,11.280000,214.100006,168.990005,104.650002,303.190002,546.760010,226.679993,60.610001,...,49.740002,263.149994,28.219999,89.139999,53.849998,114.900002,136.389999,132.610001,108.379997,176.460007
1780,2024-06-28,129.389877,11.330000,210.619995,171.520004,103.910004,303.410004,555.539978,228.259995,60.450001,...,50.259998,262.140015,28.389999,89.500000,53.410000,115.120003,135.630005,132.460007,108.529999,173.360001


Now at last we'll check if there is some missing values in prices since that would be a problem

In [219]:
null_list=prices_stocks_list.isna().sum()

In [220]:
null_list[null_list!=0]

DAY     227
FOX     447
FOXA    446
dtype: int64

We can see we have problems with two companies: Fox and Dayforce. Fox has some problems due the fact that changed its ticker in March 2019; Dayforce instead is probably due to an error on the wikipedia table page where its IPO was not listed in changes to S&P500 companies' table. In fact, before 2018 it didn't even exist as a public company. We can then safely discard this two companies from our portfolio

In [221]:
prices_stocks_list.drop(['FOX', 'FOXA', 'DAY'], axis=1, inplace=True)

We can now proceed to download our prices history in a csv file to later be used:

In [222]:
prices_stocks_list.to_csv('prices_stocks_list.csv')

## Cleaning of FactSet data

S&P500's data was downloaded from FactSet, but it's best to check it and see that everything is correct; so let's load it first

In [259]:
SP500=pd.read_csv('PriceHistorySP500.csv')

In [260]:
SP500

,Price History: SP50,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,Price,CVol,Change,% Change,"Total Return (Gross, Unhedged)","Cumulative Return % (Gross, Unhedged)",Open,High,Low
2,07/01/24,"5,475.09",4228910130.84602,14.61,0.27,"11,939.38",154.84,"5,471.08","5,479.55","5,446.53"
3,06/28/24,"5,460.48",4228910130.84602,-22.39,-0.41,"11,907.15",154.15,"5,488.48","5,523.64","5,451.12"
4,06/27/24,"5,482.87",2488146131.24847,4.97,0.09,"11,954.50",155.16,"5,473.59","5,490.81","5,467.54"
...,...,...,...,...,...,...,...,...,...,...
1779,06/07/17,"2,433.14",2821840303.97606,3.81,0.16,"4,692.29",0.15,"2,432.03","2,435.28","2,424.75"
1780,06/06/17,"2,429.33",2916385880.16129,-6.77,-0.28,"4,683.96",-0.02,"2,431.92","2,436.21","2,428.12"
1781,06/05/17,"2,436.10",2703157166.43715,-2.97,-0.12,"4,697.00",0.25,"2,437.83","2,439.55","2,434.32"
1782,06/02/17,"2,439.07",2653343130.09644,9.01,0.37,"4,702.56",0.37,"2,431.28","2,440.23","2,427.71"


We can see the file was not correctly downloaded, probably due the fact that it had to be converted from .xslx to .csv; we should fix that. Moreover we have some problems with the date being inverted and many unwanted columns

In [261]:
SP500.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 
           'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True)

In [262]:
SP500.columns = ['Date', 'Price']

In [263]:
SP500.drop(index=[0, 1], inplace=True)

Now all that's left is just to reorder dates and convert them:

In [264]:
def convert2(date_time): 
    format = '%m/%d/%y'
    datetime_str = datetime.datetime.strptime(date_time, format) 
  
    return datetime_str

In [265]:
SP500['Date']=SP500['Date'].apply(convert2)

In [266]:
SP500.sort_values(by='Date', inplace=True)

In [267]:
SP500.reset_index(inplace=True)

In [268]:
SP500.drop('index', axis=1, inplace=True)

In [269]:
SP500

,Date,Price
0,2017-06-01,"2,430.06"
1,2017-06-02,"2,439.07"
2,2017-06-05,"2,436.10"
3,2017-06-06,"2,429.33"
4,2017-06-07,"2,433.14"
...,...,...
1777,2024-06-25,"5,469.30"
1778,2024-06-26,"5,477.90"
1779,2024-06-27,"5,482.87"
1780,2024-06-28,"5,460.48"


Now we have to take care of one last thing: 

In [270]:
type(SP500['Price'][0])

str

In [ ]:
warnings.filterwarnings('ignore')
for i in range(len(SP500)):
    SP500['Price'][i]=float(SP500['Price'][i].replace(',', ''))

/var/folders/y0/c69z4ct1695f_69pbyjfvw3h0000gn/T/ipykernel_28802/3124042249.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  SP500['Price'][i]=float(SP500['Price'][i].replace(',', ''))
/var/folders/y0/c69z4ct1695f_69pbyjfvw3h0000gn/T/ipyk

Now the SP500 Dataset is complete and correct too

In [252]:
SP500.to_csv('SP500_prices.csv')